Arboles de decisiones para predecir si una persona sufrirá de un derrame cerebral
1.Leer el archivo stroke.csv

In [1]:
#----------------------------------------------------------
# Se importan las librerias
#----------------------------------------------------------

import numpy as np
import pandas as pd
import sklearn
import warnings

In [11]:
#----------------------------------------------------------
# Url de los datos
#----------------------------------------------------------

url = ("https://raw.githubusercontent.com/Yurani1143/Informe_Inteligencia_Artificial/main/stroke.csv")

#----------------------------------------------------------
# Se cargan los datos
#----------------------------------------------------------
datos = pd.read_csv(url, sep=",")
datos.columns= ["sexo","edad","hipertension","enfermedades_cardiacas","casado","tipo_trabajo","tipo_residencia","nivel_glucosa_promedio","indice_masa_corporal","estado_tabaquismo","derrame_cerebral"]
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   sexo                    700 non-null    object 
 1   edad                    700 non-null    float64
 2   hipertension            700 non-null    int64  
 3   enfermedades_cardiacas  700 non-null    int64  
 4   casado                  700 non-null    object 
 5   tipo_trabajo            700 non-null    object 
 6   tipo_residencia         700 non-null    object 
 7   nivel_glucosa_promedio  700 non-null    float64
 8   indice_masa_corporal    651 non-null    float64
 9   estado_tabaquismo       700 non-null    object 
 10  derrame_cerebral        700 non-null    int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 60.3+ KB


2. Seleccionar aleatoriamente el 80% del conjunto de datos para entrenar y el 20% restante para las pruebas

In [12]:
#----------------------------------------------------------
# Libreria sklearn
#----------------------------------------------------------
from sklearn.model_selection import train_test_split 

N=len(datos) # cantidad de datos
cTrain=int(N*0.8) # 80% para entrenar y 20% para probar
cTest=N-cTrain 

print(N,cTrain,cTest)# cantidad de: datos completos, datos entrenados, datos para probar

train_data,test_data= sklearn.model_selection.train_test_split(datos, train_size=cTrain, test_size=cTest)

700 560 140


700 es el número de observaciones en la base de datos

560 es el 80% de las observaciones de la base de datos que se van a utilizar para entrenar los datos

140 es el 20% de las observaciones de la base de datos que se van a utilizar para probar el modelo de predicción

In [13]:
#----------------------------------------------------------
# Dimensiones de los datos de entrenamiento
#----------------------------------------------------------
train_data.shape

(560, 11)

3. Utilizar una estrategia para normalizar los datos

Pipeline para los atributos categóricos

In [14]:
#----------------------------------------------------------
#Librerias sklearn
#----------------------------------------------------------
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# 5 atributos categóricos
cat_attribs = ['sexo','casado','tipo_trabajo','tipo_residencia','estado_tabaquismo']

cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False))
    ])

Pipeline para los atributos númericos

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# 5 atributos numéricos
num_attribs = ['edad','hipertension','enfermedades_cardiacas','nivel_glucosa_promedio','indice_masa_corporal']

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())

    ])

Pipeline completo

In [16]:
#----------------------------------------------------------
#Libreria sklearn
#----------------------------------------------------------
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

In [17]:
#----------------------------------------------------------
# Se transforman los datos entrenados 
# Variables independientes
#----------------------------------------------------------
X_train = full_pipeline.fit_transform(train_data)
X_train.shape

c:\Users\kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(560, 20)

In [18]:
X_train[0,:]

array([-1.18996717, -0.40227568, -0.3266793 , -0.48917712, -0.65385633,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  1.        ,  0.        ,  0.        ])

In [19]:
#----------------------------------------------------------
# Variable dependiente -> derrame_cerebral
#----------------------------------------------------------

y_train = train_data["derrame_cerebral"]
y_train

588    0
674    0
646    0
658    0
314    0
      ..
563    0
434    0
666    0
351    0
613    0
Name: derrame_cerebral, Length: 560, dtype: int64

In [20]:
#----------------------------------------------------------
# Se transforman los datos de prueba 
# Variables independientes
#--------------------------------------------------------
X_test = full_pipeline.transform(test_data) 
X_test

array([[ 0.30937074, -0.40227568, -0.3266793 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.26527256, -0.40227568, -0.3266793 , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.17707622,  2.48585743,  3.06110607, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.10313786, -0.40227568, -0.3266793 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.30937074, -0.40227568,  3.06110607, ...,  0.        ,
         0.        ,  1.        ],
       [-0.74898543, -0.40227568,  3.06110607, ...,  1.        ,
         0.        ,  0.        ]])

In [21]:
#----------------------------------------------------------
# Variable dependiente -> derrame_cerebral
#----------------------------------------------------------
y_test = test_data["derrame_cerebral"]
y_test

120    1
367    0
217    1
107    1
224    1
      ..
88     1
129    1
112    1
97     1
342    0
Name: derrame_cerebral, Length: 140, dtype: int64

4. Configurar los hiperparámetros del árbol de decisión de la siguiente manera: criterion=gini, splitter=best,
y random_state=123. Obtener 10 árboles de decisión que resultan de modificar el hiperparámetro
max_depth desde 5 hasta 50 con incrementos de 5

In [23]:
#----------------------------------------------------------
#Librerias sklearn.tree y sklearn.model_selection
#----------------------------------------------------------
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


scores=[]
scor=[]
decision_tree = []
accuracies = []
max_depths = range(5, 55, 5)

for max_depth in max_depths: 
    model_tree = DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=max_depth, random_state=123)
    model_tree.fit(X_train, y_train)
    y_pred = model_tree.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred) 
    accuracies.append(accuracy)
    decision_tree.append(model_tree.score(X_test, y_test))
    scores = cross_val_score(model_tree, X_train, y_train, cv=5,scoring='accuracy') 
    scor.append(scores.mean()) 
    print("----------------------------------")
    print("Puntajes de la validación cruzada:")
    print(scores)
    print(f"Promedio de los puntajes con max_depth={max_depth}","->" ,round(scores.mean(),4))
    print("----------------------------------")
    
#----------------------------------------------------------   
# El índice del árbol con mayor exactitud
best_tree_index = np.argmax(scor)
#----------------------------------------------------------

#----------------------------------------------------------
# Se imprime los hiperparámetros del árbol con mayor precisión
print("Los hiperparámetros del árbol con mayor precisión son:")
print("max_depth:", max_depths[best_tree_index])
print("criterion: gini")
print("splitter: best")
print("random_state: 123")

----------------------------------
Puntajes de la validación cruzada:
[0.8125     0.83035714 0.75       0.76785714 0.73214286]
Promedio de los puntajes con max_depth=5 -> 0.7786
----------------------------------
----------------------------------
Puntajes de la validación cruzada:
[0.75       0.75       0.74107143 0.73214286 0.65178571]
Promedio de los puntajes con max_depth=10 -> 0.725
----------------------------------
----------------------------------
Puntajes de la validación cruzada:
[0.72321429 0.70535714 0.74107143 0.74107143 0.64285714]
Promedio de los puntajes con max_depth=15 -> 0.7107
----------------------------------
----------------------------------
Puntajes de la validación cruzada:
[0.72321429 0.70535714 0.74107143 0.74107143 0.64285714]
Promedio de los puntajes con max_depth=20 -> 0.7107
----------------------------------
----------------------------------
Puntajes de la validación cruzada:
[0.72321429 0.70535714 0.74107143 0.74107143 0.64285714]
Promedio de los pun

5. Incluya en el notebook una tabla con el accuracy para los 10 árboles del punto anterior

In [24]:
#----------------------------------------------------------
#Libreria pandas
#----------------------------------------------------------
import pandas as pd

# Mostrar el accuracy de los 10 arboles construidos
accuracies_CV = pd.DataFrame({"max_depth": range(5, 55, 5), "accuracy": scor})
display(accuracies_CV)

,max_depth,accuracy
0,5,0.778571
1,10,0.725000
2,15,0.710714
3,20,0.710714
4,25,0.710714
5,30,0.710714
6,35,0.710714
7,40,0.710714
8,45,0.710714
9,50,0.710714


6. Repita el mismo procedimiento del punto 4 usando como hiperparámetros criterion=entropy, splitter=best,
random_state=123, y variando el hiperparámetro max_depth desde 5 hasta 50 con incrementos de 5

In [25]:
#----------------------------------------------------------
#Librerias sklearn.tree y sklearn.model_selection
#----------------------------------------------------------
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


scores_2=[]
scor_2 =[]
decision_tree_2 = []
accuracies_p7= []
max_depths = range(5, 55, 5)

for max_depth in max_depths: 
    model_tree_2 = DecisionTreeClassifier(criterion="entropy", splitter="best", max_depth=max_depth, random_state=123)
    model_tree_2.fit(X_train, y_train)
    accuracy_7 = accuracy_score(y_test, y_pred) # exactitud de cada árbol
    accuracies_p7.append(accuracy_7)
    decision_tree_2.append(model_tree_2.score(X_test, y_test))
    scores_2 = cross_val_score(model_tree_2, X_train, y_train, cv=5,scoring='accuracy')
    scor_2.append(scores_2.mean()) # promedio de los scores
    print("----------------------------------")
    print("Puntajes de la validación cruzada:")
    print(scores_2)
    print(f"Promedio de los puntajes con max_depth={max_depth}","->" ,round(scores_2.mean(),4))
    print("----------------------------------")

#----------------------------------------------------------   
# El índice del árbol con mayor exactitud
best_tree_index_2 = np.argmax(scor_2)
#----------------------------------------------------------

#----------------------------------------------------------
# Se imprime los hiperparámetros del árbol con mayor precisión
print("Los hiperparámetros del árbol con mayor precisión son:")
print("max_depth:", max_depths[best_tree_index_2])
print("criterion: entropy")
print("splitter: best")
print("random_state: 123")

----------------------------------
Puntajes de la validación cruzada:
[0.74107143 0.78571429 0.78571429 0.72321429 0.73214286]
Promedio de los puntajes con max_depth=10 -> 0.7536
----------------------------------
----------------------------------
Puntajes de la validación cruzada:
[0.75       0.75892857 0.75892857 0.6875     0.67857143]
Promedio de los puntajes con max_depth=20 -> 0.7268
----------------------------------
----------------------------------
Puntajes de la validación cruzada:
[0.75       0.75892857 0.75892857 0.6875     0.67857143]
Promedio de los puntajes con max_depth=30 -> 0.7268
----------------------------------
----------------------------------
Puntajes de la validación cruzada:
[0.75       0.75892857 0.75892857 0.6875     0.67857143]
Promedio de los puntajes con max_depth=40 -> 0.7268
----------------------------------
----------------------------------
Puntajes de la validación cruzada:
[0.75       0.75892857 0.75892857 0.6875     0.67857143]
Promedio de los p

7. Incluya en el notebook una tabla con el accuracy para los 10 árboles del punto anterior

In [26]:
#----------------------------------------------------------
#Libreria pandas
#----------------------------------------------------------
import pandas as pd

# Mostrar el accuracy de los 10 arboles construidos 
accuracies_2 = pd.DataFrame({"max_depth": range(10, 110, 10), "accuracy": scor_2})
display(accuracies_2)

,max_depth,accuracy
0,10,0.753571
1,20,0.726786
2,30,0.726786
3,40,0.726786
4,50,0.726786
5,60,0.726786
6,70,0.726786
7,80,0.726786
8,90,0.726786
9,100,0.726786
